In [ ]:
# Check Pytorch installation

import torch
print(torch.__version__, torch.cuda.is_available())

from multiprocessing import Process

if __name__ == '__main__':    
    # Check MMAction2 installation
    import mmaction
    print(mmaction.__version__)

    import mmcv
    print(mmcv.__version__)

    # Check MMCV installation
    from mmcv.ops import get_compiling_cuda_version, get_compiler_version
    print(get_compiling_cuda_version())
    print(get_compiler_version())

    # Check MMEngine installation
    from mmengine.utils.dl_utils import collect_env
    print(collect_env())

    from mmaction.apis import inference_recognizer, init_recognizer
    from mmengine import Config

    from operator import itemgetter

    import os.path as osp
    import mmengine
    from mmengine.runner import Runner

    cfg = "path to config file"
    cfg = Config.fromfile(cfg)

    # Set up working dir to save files and logs.
    cfg.work_dir = './exps/mvitv2_regular_recognition'

    # Create work_dir
    mmengine.mkdir_or_exist(osp.abspath(cfg.work_dir))

    # The original learning rate (LR) is set for 8-GPU training.
    # We divide it by 8 since we only use one GPU.
    cfg.train_dataloader.batch_size = cfg.train_dataloader.batch_size // 4
    cfg.val_dataloader.batch_size = cfg.val_dataloader.batch_size // 4
    cfg.test_dataloader.batch_size = 1
    cfg.optim_wrapper.optimizer.lr = cfg.optim_wrapper.optimizer.lr / 8
    cfg.train_cfg.max_epochs = 100


    # We can initialize the logger for training and have a look
    # at the final config used for training
    print(f'Config:\n{cfg.pretty_text}')

    # Initialize the recognizer
    model = init_recognizer(cfg, checkpoint=None, device='cuda:0')

    # build the runner from config
    runner = Runner.from_cfg(cfg)

    # start training
    runner.train()